## Programados

In [ ]:
!pip install schedule

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime
import pytz

# Configuración inicial
ACCESS_TOKEN = "EAAUrP6ZAqWeYBO2ghoxlqjAAkGvN7qybNYfDJqSghwZBhkdfEbJD2icXBXdOwBZBIZBkKtigL5zH7cGfCbxNQkezzXEnyAm090BWobWMRvIRsyvO8XXZBBc42c0090HNuxLynbwlsGNildpRhbZBKlkHYWGxMpVgTPOcVKqaDtQebzrc3XJnxJJL0pQ4fr87a7GdoUYHYt95ZCMUnYZC5G4ZD"
WABA_ID = "379380888591902"
PHONE_NUMBER_ID = "371453729380075"
TEMPLATE_NAME = "hello_world"
EXCEL_FILE = "/content/drive/MyDrive/opti/PRUEBA1.xlsx"

WHATSAPP_API_URL = f"https://graph.facebook.com/v17.0/{PHONE_NUMBER_ID}/messages"
HEADERS = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Content-Type": "application/json"
}

# Función para obtener detalles de la plantilla
def get_template_details(template_name):
    url = f"https://graph.facebook.com/v17.0/{WABA_ID}/message_templates"
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        for template in data.get("data", []):
            if template["name"] == template_name:
                return template
        print("❌ Plantilla no encontrada.")
    else:
        print("❌ Error al obtener plantilla:", response.status_code, response.text)
    return None

# Función para solicitar parámetros faltantes
def prompt_for_component(component_type, format_type):
    if format_type == "IMAGE":
        return input("🔹 Ingresa la URL de la imagen: ").strip()
    elif format_type == "VIDEO":
        return input("🔹 Ingresa la URL del video: ").strip()
    elif format_type == "DOCUMENT":
        return input("🔹 Ingresa la URL del documento: ").strip()
    elif format_type == "LOCATION":
        lat = input("🔹 Ingresa la latitud de la ubicación: ").strip()
        lon = input("🔹 Ingresa la longitud de la ubicación: ").strip()
        return {"latitude": lat, "longitude": lon}
    return None

# Función para construir el mensaje dinámicamente
def build_message(template_data, recipient, name, additional_params):
    components = []
    for component in template_data.get("components", []):
        if component["type"] == "FOOTER":
            continue

        comp = {"type": component["type"], "parameters": []}

        # HEADER
        if component["type"] == "HEADER":
            format_type = component.get("format")
            if format_type == "TEXT":
                pass
            elif format_type in ["IMAGE", "VIDEO", "DOCUMENT"]:
                url = additional_params.get("header_url")
                if not url:
                    raise ValueError(f"URL faltante para el componente HEADER de tipo {format_type}")
                comp["parameters"].append({
                    "type": format_type.lower(),
                    format_type.lower(): {"link": url}
                })
            elif format_type == "LOCATION":
                location = additional_params.get("header_location")
                if not location:
                    raise ValueError("Coordenadas faltantes para el componente HEADER de tipo LOCATION")
                comp["parameters"].append({"type": "location", "location": location})

        # BODY
        elif component["type"] == "BODY":
            param_count = component["text"].count("{{")
            for i in range(param_count):
                comp["parameters"].append({"type": "text", "text": name})

        # Agregar componente si tiene parámetros
        if comp["parameters"]:
            components.append(comp)

        # Manejar estructura del campo "language"
    if isinstance(template_data["language"], dict):
        language_code = template_data["language"]["code"]
    else:
        language_code = template_data["language"]

    return {
        "messaging_product": "whatsapp",
        "to": recipient,
        "type": "template",
        "template": {
            "name": TEMPLATE_NAME,
            "language": {"code": language_code},
            "components": components
        }
    }

# Función para solicitar parámetros dinámicos de la plantilla
def get_additional_params(template_data):
    additional_params = {}
    for component in template_data.get("components", []):
        if component["type"] == "HEADER" and component.get("format") != "TEXT":
            format_type = component.get("format")
            result = prompt_for_component("HEADER", format_type)
            if format_type in ["IMAGE", "VIDEO", "DOCUMENT"]:
                additional_params["header_url"] = result
            elif format_type == "LOCATION":
                additional_params["header_location"] = result
    return additional_params

# Función para enviar mensajes masivos
def send_bulk_messages(file_path, template_data, additional_params):
    df = pd.read_excel(file_path)
    if not {'Nombre', 'Celular'}.issubset(df.columns):
        raise ValueError("El archivo Excel debe contener las columnas 'Nombre' y 'Celular'.")

    phone_numbers = df['Celular'].apply(lambda x: str(int(float(x))).strip()).tolist()
    first_names = df['Nombre'].astype(str).tolist()
    errors = []

    for index, phone_number in enumerate(phone_numbers):
        message = build_message(template_data, phone_number, first_names[index], additional_params)
        try:
            response = requests.post(WHATSAPP_API_URL, headers=HEADERS, json=message)
            if response.status_code == 200:
                print(f"✅ Mensaje enviado a {phone_number} ({first_names[index]})")
            else:
                print(f"❌ Error al enviar mensaje a {phone_number}: {response.text}")
                errors.append(response.text)
        except Exception as e:
            errors.append(f"Error con {phone_number}: {str(e)}")
    return errors

# Función para programar mensajes
def schedule_messages(file_path, template_data, schedule_time):
    # Verificar si la fecha y hora ingresadas son válidas
    local_timezone = pytz.timezone("America/Bogota")
    current_time = datetime.now(local_timezone)
    if schedule_time <= current_time:
        print("❌ La fecha y hora ingresadas ya pasaron. No se pueden programar los mensajes.")
        return

    print(f"⌛ Mensajes programados para enviarse a las {schedule_time.strftime('%Y-%m-%d %H:%M %Z')}")

    # Solicitar parámetros faltantes antes de la espera
    additional_params = get_additional_params(template_data)

    # Esperar hasta el momento programado
    while True:
        now = datetime.now(local_timezone)
        if now >= schedule_time:
            errors = send_bulk_messages(file_path, template_data, additional_params)
            if errors:
                print("❌ Se encontraron errores durante el envío:")
                for error in errors:
                    print(error)
            else:
                print("✅ ¡Todos los mensajes se enviaron correctamente!")
            break
        time.sleep(1)

# Función para obtener la fecha y hora desde la terminal
def get_schedule_time():
    local_timezone = pytz.timezone("America/Bogota")
    while True:
        schedule_time = input("🔹 Ingresa la fecha y hora para programar los mensajes (YYYY-MM-DD HH:MM): ").strip()
        try:
            naive_time = datetime.strptime(schedule_time, "%Y-%m-%d %H:%M")
            localized_time = local_timezone.localize(naive_time)
            return localized_time
        except ValueError:
            print("❌ Formato inválido. Inténtalo de nuevo.")

# Flujo principal
if __name__ == "__main__":
    print("📤 Obteniendo detalles de la plantilla...")
    template_data = get_template_details(TEMPLATE_NAME)
    if template_data:
        print("✅ Plantilla obtenida correctamente.")
        schedule_time = get_schedule_time()
        schedule_messages(EXCEL_FILE, template_data, schedule_time)
    else:
        print("❌ No se pudo obtener la información de la plantilla.")


📤 Obteniendo detalles de la plantilla...
✅ Plantilla obtenida correctamente.
🔹 Ingresa la fecha y hora para programar los mensajes (YYYY-MM-DD HH:MM): 2025-01-09 19:07
⌛ Mensajes programados para enviarse a las 2025-01-09 19:07 -05
✅ Mensaje enviado a 573004710977 (Zhari)
✅ Mensaje enviado a 573023023682 (Emanuel)
✅ ¡Todos los mensajes se enviaron correctamente!


In [ ]:
def validate_media_url(url):
    """Valida que la URL del medio sea accesible"""
    try:
        response = requests.head(url, allow_redirects=True)
        if response.status_code == 200:
            return True
        else:
            print(f"❌ La URL no es accesible. Código de estado: {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ Error al validar la URL: {e}")
        return False
validate_media_url("https://www.dropbox.com/scl/fi/kklhz9dy906toictt1yl9/Video.mp4?rlkey=fflot3uqr4jdfgzuaap5ipkm6&st=7lqy7iv6&dl=0")

True

## **Envio de plantillas a Meta**

In [ ]:
import requests
import json

# Configuración inicial
ACCESS_TOKEN = "EAAUrP6ZAqWeYBO2ghoxlqjAAkGvN7qybNYfDJqSghwZBhkdfEbJD2icXBXdOwBZBIZBkKtigL5zH7cGfCbxNQkezzXEnyAm090BWobWMRvIRsyvO8XXZBBc42c0090HNuxLynbwlsGNildpRhbZBKlkHYWGxMpVgTPOcVKqaDtQebzrc3XJnxJJL0pQ4fr87a7GdoUYHYt95ZCMUnYZC5G4ZD"
BUSINESS_ACCOUNT_ID = "379380888591902"  # ID de la cuenta empresarial
GRAPH_API_VERSION = "v18.0"  # Versión de la API
TEMPLATE_NAME = "panaderia"  # Nombre único

# URL de creación de plantilla
url = f"https://graph.facebook.com/{GRAPH_API_VERSION}/{BUSINESS_ACCOUNT_ID}/message_templates"

# Payload con plantilla más completa
payload = {
    "name": TEMPLATE_NAME,
    "category": "MARKETING",
    "language": "es",
    "components": [
        # Encabezado con texto
        {
            "type": "HEADER",
            "format": "TEXT",
            "text": "¡Promoción Especial!"
        },
        # Cuerpo del mensaje
        {
            "type": "BODY",
            "text": "Hola {{1}}, obtén un descuento exclusivo del {{2}} en tus próximas gafas. ¡No te lo pierdas!"
        },
        # Pie de página
        {
            "type": "FOOTER",
            "text": "Oferta válida hasta el 30 de septiembre."
        },
        # Botones interactivos
        {
            "type": "BUTTONS",
            "buttons": [
                {
                    "type": "URL",
                    "text": "Ver promoción",
                    "url": "https://optichat.co/"
                },
                {
                    "type": "QUICK_REPLY",
                    "text": "¡Quiero más info!"
                }
            ]
        }
    ]
}

# Encabezados HTTP
headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Content-Type": "application/json"
}

# Enviar solicitud POST
response = requests.post(url, headers=headers, json=payload)

# Verificar la respuesta
if response.status_code == 200:
    print("✅ ¡Plantilla creada exitosamente!")
else:
    print("❌ Error al crear la plantilla:")
    print(response.json())


✅ ¡Plantilla creada exitosamente!


In [ ]:
import requests
import json

# Configuración inicial
ACCESS_TOKEN = "EAAUrP6ZAqWeYBO2ghoxlqjAAkGvN7qybNYfDJqSghwZBhkdfEbJD2icXBXdOwBZBIZBkKtigL5zH7cGfCbxNQkezzXEnyAm090BWobWMRvIRsyvO8XXZBBc42c0090HNuxLynbwlsGNildpRhbZBKlkHYWGxMpVgTPOcVKqaDtQebzrc3XJnxJJL0pQ4fr87a7GdoUYHYt95ZCMUnYZC5G4ZD"
BUSINESS_ACCOUNT_ID = "379380888591902"  # ID de la cuenta empresarial
GRAPH_API_VERSION = "v18.0"  # Versión de la API
TEMPLATE_NAME = "panaderia_nueva"  # Nombre único para evitar rechazo por duplicidad

# URL de creación de plantilla
url = f"https://graph.facebook.com/{GRAPH_API_VERSION}/{BUSINESS_ACCOUNT_ID}/message_templates"

# Payload corregido
payload = {
    "name": TEMPLATE_NAME,
    "category": "MARKETING",
    "language": "es",
    "components": [
        # Encabezado con texto
        {
            "type": "HEADER",
            "format": "TEXT",
            "text": "¡Tenemos algo especial para ti!"
        },
        # Cuerpo del mensaje
        {
            "type": "BODY",
            "text": "Hola {{1}}, queremos agradecerte por tu preferencia. Disfruta de un beneficio exclusivo del {{2}}%. ¿Te gustaría saber más?"
        },
        # Pie de página
        {
            "type": "FOOTER",
            "text": "Consulta términos y condiciones en nuestro sitio web."
        },
        # Botón de URL corregido (sin QUICK_REPLY inicialmente)
        {
            "type": "BUTTONS",
            "buttons": [
                {
                    "type": "URL",
                    "text": "Ver más detalles",
                    "url": "https://optichat.co/"
                }
            ]
        }
    ]
}

# Encabezados HTTP
headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Content-Type": "application/json"
}

# Enviar solicitud POST
response = requests.post(url, headers=headers, json=payload)

# Verificar la respuesta
if response.status_code == 200:
    print("✅ ¡Plantilla corregida y enviada exitosamente!")
else:
    print("❌ Error al crear la plantilla:")
    print(response.json())


✅ ¡Plantilla corregida y enviada exitosamente!


In [ ]:
import requests
import json

# Configuración inicial
ACCESS_TOKEN = "EAAUrP6ZAqWeYBO2ghoxlqjAAkGvN7qybNYfDJqSghwZBhkdfEbJD2icXBXdOwBZBIZBkKtigL5zH7cGfCbxNQkezzXEnyAm090BWobWMRvIRsyvO8XXZBBc42c0090HNuxLynbwlsGNildpRhbZBKlkHYWGxMpVgTPOcVKqaDtQebzrc3XJnxJJL0pQ4fr87a7GdoUYHYt95ZCMUnYZC5G4ZD"
BUSINESS_ACCOUNT_ID = "379380888591902"  # ID de la cuenta empresarial
GRAPH_API_VERSION = "v18.0"  # Versión de la API
TEMPLATE_NAME = "panaderia_info"  # Nuevo nombre para evitar duplicados rechazados

# URL de creación de plantilla
url = f"https://graph.facebook.com/{GRAPH_API_VERSION}/{BUSINESS_ACCOUNT_ID}/message_templates"

# Payload corregido con un enfoque más neutro
payload = {
    "name": TEMPLATE_NAME,
    "category": "UTILITY",  # Cambiado de "MARKETING" a "UTILITY"
    "language": "es",
    "components": [
        # Encabezado con texto informativo
        {
            "type": "HEADER",
            "format": "TEXT",
            "text": "¡Un mensaje especial para ti!"
        },
        # Cuerpo del mensaje sin referencias comerciales directas
        {
            "type": "BODY",
            "text": "Hola {{1}}, queremos compartir una información importante contigo. Si necesitas más detalles, dime y con gusto te ayudaré."
        },
        # Pie de página con un mensaje neutro
        {
            "type": "FOOTER",
            "text": "Estamos aquí para cualquier consulta."
        }
    ]
}

# Encabezados HTTP
headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Content-Type": "application/json"
}

# Enviar solicitud POST
response = requests.post(url, headers=headers, json=payload)

# Verificar la respuesta
if response.status_code == 200:
    print("✅ ¡Plantilla corregida y enviada exitosamente!")
else:
    print("❌ Error al crear la plantilla:")
    print(response.json())


✅ ¡Plantilla corregida y enviada exitosamente!


## WEBHOOK

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Token que colocaste en Meta
VERIFY_TOKEN = "mi_token_de_verificacion"  # Coloca aquí el token que tú defines

@app.route('/webhook', methods=['GET', 'POST'])
def webhook():
    if request.method == 'GET':
        # Meta enviará GET con parámetros para verificación
        mode = request.args.get('hub.mode')
        token = request.args.get('hub.verify_token')
        challenge = request.args.get('hub.challenge')

        # Compara el token proporcionado por Meta con el tuyo
        if mode == 'subscribe' and token == VERIFY_TOKEN:
            print("✅ Verificación exitosa.")
            return challenge, 200
        else:
            print("❌ Falló la verificación.")
            return "Error: Verificación fallida", 403

    elif request.method == 'POST':
        # Meta envía respuestas como POST aquí
        data = request.json
        print("📨 Webhook recibido:", data)

        # Procesa el mensaje
        if 'messages' in data['entry'][0]['changes'][0]['value']:
            message = data['entry'][0]['changes'][0]['value']['messages'][0]
            sender = message['from']
            body = message['text']['body']
            print(f"Mensaje de {sender}: {body}")

        return jsonify({"status": "Mensaje recibido"}), 200

if __name__ == '__main__':
    app.run(port=8080, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


Address already in use
Port 8080 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/serving.py", line 759, in __init__
    self.server_bind()
  File "/usr/lib/python3.10/http/server.py", line 137, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/usr/lib/python3.10/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-709733d5c68a>", line 39, in <cell line: 38>
    app.run(port=8080, debug=True)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 625, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/serving.py", lin

TypeError: object of type 'NoneType' has no len()